# Deskripsi Proyek

Proyek ini bertujuan untuk membangun pipeline machine learning untuk memprediksi apakah seorang mahasiswa akan mendapatkan penempatan atau placement berdasarkan data numerik seperti IPK dan kemampuan soft skill.

Pipeline ini mencakup komponen-komponen:
- Ingest data
- Validasi skema
- Transformasi fitur
- Pelatihan model
- Evaluasi model
- Pushing model akhir


# Import Library
Merupakan tahapan paling awal yang berguna untuk melakukan import library yang akan digunakan

In [1]:
import os
import tensorflow as tf
import tensorflow_model_analysis as tfma 
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.dsl.components.common.resolver import Resolver 
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 

# Set Variable
Merupakan setting awal untuk menetapkan beberapa variabel global

In [2]:
PIPELINE_NAME = "pipeline"
SCHEMA_PIPELINE_NAME = "schema"
PIPELINE_ROOT = 'naufal_azmi_ginting-pipeline' #os.path.join('naufal_azmi_ginting-pipeline', PIPELINE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'metadata', 'metadata.db')
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, 'serving_model')
DATA_ROOT = "data"

In [3]:
METADATA_PATH

'naufal_azmi_ginting-pipeline\\metadata\\metadata.db'

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# ExampleGen
Membaca dataset dari file CSV dan mengonversinya menjadi format TFRecord.

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

# StatisticGen
Menghasilkan statistik deskriptif dari data untuk keperluan validasi dan deteksi anomali.

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

# SchemaGen
Menghasilkan skema data berdasarkan statistik dari data training.

In [9]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Academic_Performance',INT,required,,-
'CGPA',FLOAT,required,,-
'Communication_Skills',INT,required,,-
'Extra_Curricular_Score',INT,required,,-
'IQ',INT,required,,-
'Internship_Experience',INT,required,,-
'Placement',INT,required,,-
'Prev_Sem_Result',FLOAT,required,,-
'Projects_Completed',INT,required,,-


# ExampleValidator
Mengevaluasi validitas data dan mendeteksi nilai outlier atau missing berdasarkan skema.

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Transform
Fitur-fitur numerik di-scale menggunakan tft.scale_to_z_score() agar memiliki distribusi normal (mean 0, std 1). Label "Placement" tidak diubah.

In [13]:
TRANSFORM_MODULE_FILE = "transform.py"

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft

FEATURE_KEYS = [
    "IQ", "Prev_Sem_Result", "CGPA", "Academic_Performance",
    "Internship_Experience", "Extra_Curricular_Score",
    "Communication_Skills", "Projects_Completed"
]

LABEL_KEY = "Placement"

def preprocessing_fn(inputs):
    outputs = {}
    
    for key in FEATURE_KEYS:
        outputs[key] = tft.scale_to_z_score(inputs[key])
    outputs[LABEL_KEY] = inputs[LABEL_KEY]
    return outputs

Overwriting transform.py


In [15]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\e3a26607f4de4d13848aace8ef6a51c6\assets


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\e3a26607f4de4d13848aace8ef6a51c6\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

# Trainer
Model neural network sederhana dibangun menggunakan Keras Sequential. Model ini dilatih menggunakan data yang sudah ditransformasi sebelumnya.

Proses pelatihan menggunakan Trainer yang menyimpan model lengkap beserta preprocessing layer, sehingga saat serving nanti input tetap diproses sesuai transformasi training.

In [16]:
MODEL_FILE = "model.py"

In [17]:
%%writefile {MODEL_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

# Keys
LABEL_KEY = "Placement"
FEATURE_KEYS = [
                    "IQ","Prev_Sem_Result",
                    "CGPA","Academic_Performance",
                    "Internship_Experience","Extra_Curricular_Score",
                    "Communication_Skills","Projects_Completed"
                ]

def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32):
    transformed_feature_spec = tf_transform_output.transformed_feature_spec().copy()

    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=gzip_reader_fn,
        label_key=LABEL_KEY,
        num_epochs=num_epochs,
    )
    return dataset

def model_builder():
    inputs = {
        k: tf.keras.Input(shape=(1,), name=k)
        for k in FEATURE_KEYS
    }
    x = tf.keras.layers.Concatenate()(list(inputs.values()))
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.BinaryAccuracy()])
    model.summary()
    return model

def _get_serve_tf_examples_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)
    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs):
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)

    model = model_builder()
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')
    model.fit(train_set,
              validation_data=val_set,
              epochs=10,
              steps_per_epoch=100,
              validation_steps=50,
              callbacks=[tensorboard_callback])
    signatures = {
        'serving_default':
            _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                tf.TensorSpec(shape=[None], dtype=tf.string, name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting model.py


In [18]:
trainer = Trainer(
    module_file=MODEL_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50)
)
interactive_context.run(trainer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 IQ (InputLayer)                [(None, 1)]          0           []                               
                                                                                                  
 Prev_Sem_Result (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 CGPA (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 Academic_Performance (InputLay  [(None, 1)]         0           []                               
 er)                                                                                          

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: naufal_azmi_ginting-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

# Resolver
Untuk mengambil model terbaik dan terbaru dari hasil pipeline sebelumnya

In [19]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

# Evaluator
Komponen Evaluator mengevaluasi performa model pada data validasi. Evaluasi dilakukan berdasarkan metrik akurasi dan metrik lain seperti false positive atau false negative.

In [20]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Placement')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [21]:
from pathlib import Path

transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
assert Path(transform_graph_uri).exists(), "Transform graph path tidak ditemukan."

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [22]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Pusher
Model dengan performa baik dipilih dan dikirim ke direktori serving. Model ini telah siap digunakan dalam aplikasi produksi

In [23]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))